# Vector Calculus and image processing

**Objectives:** In this notebook you will
- see differential operators acting on images
- learn about a simple partial differential equation
- discretize and numerically approximate the solutions of this PDE
- make a connection to energy minimization and Gradient descent
- apply this to image denoising and reconstruction

The following box you have to run once at the beginning of each session in order to load all packages that are needed.

In [1]:
using Images, Gadfly, TestImages, ImageView, Colors

ArgumentError: ArgumentError: Package Images not found in current path:
- Run `import Pkg; Pkg.add("Images")` to install the Images package.


# Images as scalar fields

Gray scale images are scalar fields $f\colon \mathbb{R}^2\to \mathbb{R}$. A number, usually between 0 (totally black) and 255 (totally white) (8bits), encodes the gray level at a certain position. Thus one may also think of images as landscapes. A digital image is just a matrix. More precisely, a rectangle of certain size is discretized and given via an $M\times N$ matrix of pixel positions. The entries of the matrix encode the gray levels of the pixels. So a typical digital gray scale image is a function $f\colon\{1,\ldots,M\}\times\{1,\ldots,N\}\to\{0,\ldots,255\}$ where $f(m,n)$ is the gray level of the pixel at position $(m,n)$. Let us first read a jpeg image and tranform it into a matrix of gray level entries.

In [2]:
#im = convert(Array{Float64}, testimage("cameraman"))
im = convert(Array{Float64}, Gray.(load("huggingzebras.jpg")))
M, N = size(im)
Gray.(im)

UndefVarError: UndefVarError: load not defined



# The Gradient

The main question on zebras is: Are they black with white stripes, or are they white with black stripes? But mathematics has nothing to say about this! Instead we do the following:

Discrete derivatives are computed as finite difference approximations of derivatives. For instance, as partial derivatives in the coordinate directions one might consider

$$ \delta_1f(n_1,n_2) := f(n_1+1,n_2) - f(n_1,n_2) $$
$$ \delta_2f(n_1,n_2) := f(n_1,n_2+1) - f(n_1,n_2) $$

One has to be careful at the boundary of the domain. You could for example consider periodic boundary conditions which corresponds to computing the indices $n_i+1$ modulo $N$, i.e. you just let $n_i+1=1$ for $n_i=N$ and $n_i-1=N$ for $n_i=1$. In our below implementation we won't even do that, but just disregard the boundary and possible artifacts created there.

A discrete gradient is

$$\nabla f(n):= \begin{bmatrix}\delta_1f(n)\\ \delta_2f(n) \\ \end{bmatrix} \in \mathbb{R}^{2}$$

i.e. the gradient maps images to vector fileds, $\nabla\colon \mathbb{R}^{N\times M}\to \mathbb{R}^{N\times M\times 2}$.

Let's compute and look at the partial derivatives in the two coordinate directions.

In [3]:
gx = zeros(M,N)
gy = zeros(M,N)
j = 1:(M-1)
k = 1:(N-1)

gx[j,k] = im[j,k.+1] - im[j,k]
gy[j,k] = im[j.+1,k] - im[j,k]

#Display matrix as grayscale image
Gray.([gx gy])

UndefVarError: UndefVarError: M not defined

Denote the Euclidean length of the gradient at a particular $(n_1,n_2)$ by

$$
|| \nabla f(n_1,n_2) || := \sqrt{(\delta_1f(n_1,n_2))^2+(\delta_2f(n_1,n_2))^2}
$$

This is dispayed next.

In [4]:
Gr_mag = sqrt.(gx.^2 + gy.^2)
Gray.(Gr_mag)

UndefVarError: UndefVarError: gx not defined

The range of the gradient magnitude is huge. And we only have 256 gray levels. So the small gradient magnitudes are not even visible. We can look at this on a log scale. Then for example the tree in the background becomes visible. 

In [5]:
# View Gr_mag after applying a logarithm ... 

# Gradient descent

We know that the gradient points into the direction of the steepest ascent and that its length measures how steep the ascent in this direction is. If we are anywhere in a lanscape of a scalar field, we can compute the gradient and walk a step into this direction towards a maximum or a step into the opposite direction towards a minimum.
Note that, in general, at every point the direction and magnitude of the gradient changes.


**Example:**  Consider the quadratic dwell

$$ g(x,y) = \frac{1}{2}(\alpha x^2 +y^2 )$$

for $x,y\in\mathbb{R}$. Of course, we know that the minimum of this function is at $(0,0)$. Let's see what a gradient descent does. Since
we have a formula for $g$ we can compute the gradient analytically

$$ \nabla g (x,y) = \begin{bmatrix}\alpha x\\ y \\ \end{bmatrix} $$

The idea is to compute the gradient at the point we are currently at, and to walk a small step into the opposite direction. Then, compute the gradient at the new position and again walk a step into precisely to opposite of this direction. 
That is, starting from some point $p^{(0)}=(x^{(0)},y^{(0)})^\top$, iteratively compute a sequence of points $p^{(k)}=(x^{(k)},y^{(k)})^\top$ as follows

$$ p^{(k+1)} = p^{(k)} - s \nabla g (p^{(k)}) $$


But what step size $s>0$ to choose? Essentially the choices fall into three categories: the Good, the Bad, and the Ugly. If the stepsize is too small, here say $s=0.02$, the gradient descent will be very slow. Choosen in the right ballpark, the convergence is faster, say for $s=0.18$. If the step size is chosen too large, then the gradient descent may not converge to anything at all, say for $s=0.22$. It is a question adressed in numerical real analysis how to choose the step size for a particular function at hand. Describe what happens for different values of $s$.

In [6]:
X = Array(-20:0.1:20); Y = X #discretised domain for the function
alpha = 10
Z = 0.5*((alpha * (X.^2)) .+ Y'.^2) #compute all values z=g(x,y)
p = plot(x=X, y=Y, z=Z, Geom.contour(levels=40))

numiters = 30 #number of iterations in gradient descent
s = 0.18;     #choose a stepsize

# Create empty array to store the iterates
X = Array{Float64}(undef, numiters) 
Y = Array{Float64}(undef, numiters)
X[1] = 4; Y[1] = 19.8 #choose a starting point

# Gradient descent
for k = 2:numiters
    Xold, Yold = X[k-1], Y[k-1]
    X[k], Y[k] = Xold - s*alpha*Xold, Yold - s*Yold
end
push!(p, layer(x=X, y=Y, Geom.point, Geom.path))
draw(SVG(700px, 500px), p)

UndefVarError: UndefVarError: Geom not defined

The iterates may approach the minimum rather slow if the problem has bad conditions. Say, if $\alpha = 100$ then the quadratic is much like a skateboard halfpipe. With step size $s=0.02$ we converge very slow because of serious zigzagging. But for bigger $s$ we diverge. There are ways to overcome such issues, for instance by changing the step sizes throughout the iterations.

In [7]:
X = Array(-20:0.1:20); Y=X #discretised domain for the function
alpha = 100
Z = 0.5*((alpha * (X.^2)) .+ Y'.^2) #compute all values z=g(x,y)
p = plot(x=X, y=Y, z=Z, Geom.contour(levels=40))

numiters = 40 #number of iterations in gradient descent
s = 0.02;     #choose a stepsize
X = Array{Float64}(undef, numiters)
Y = Array{Float64}(undef, numiters)
X[1]=4; Y[1] = 19.8 #choose a starting point

# Gradient descent
for k=2:numiters
    Xold, Yold = X[k-1], Y[k-1]
    X[k], Y[k] = Xold - s*alpha*Xold, Yold - s*Yold
end
push!(p, layer(x=X, y=Y, Geom.point, Geom.path))
draw(SVG(700px, 500px), p)

UndefVarError: UndefVarError: Geom not defined

You probably have observed that in a complicating landscape gradient descent will in general not lead us to a global minimum: What if there are many local minima or maxima? One can get trapped in one of them. The choice of starting point can then determine to what point we converge.
Such issues cannot happen if the function you are dealing with are "convex". Also one has to think about whether a minimum exists at all.


# Divergence and Laplace operator

Above, we discretized the partial derivative by a simple forward difference, e.g. 
$ \delta_1f(n_1,n_2) = f(n_1+1,n_2) - f(n_1,n_2) $. Now, is this the approximation of the partial derivative at $(n_1,n_2)$ or at $(n_1+1,n_2)$? Well, it's not clear. Maybe it is an approximation at $(n_1+1/2,n_2)$? But there are no half pixels on the digital image. Moreover, we might as well have discretized the partial derivative with  backwards differences:

$$ \delta_1^*f(n_1,n_2) := f(n_1,n_2) - f(n_1-1,n_2) $$
$$ \delta_2^*f(n_1,n_2) := f(n_1,n_2) - f(n_1,n_2-1) $$

To compute a second derivative, we don't want to apply the forward difference twice. It seems more reasonable to do a forward and then a backwards difference, in order not to move to far away from $(n_1,n_2)$.


 - **Exercise:** Let $f,g\in\mathbb{R}^N$ and let $\delta f$ denote the forward and $\delta^* f$ the backward difference. Assuming periodic boundary conditions, show that for the Euclidean inner product we have
$$
\langle \delta f, g \rangle = -\,
\langle f,\delta^* g\rangle
$$
This can be interpreted as a discrete version of the integration by parts formula
$$
\int_0^1 f'g = - \int_0^1 fg'
$$
for smooth functions $f,g$ with $f(0)=f(1)$ and $g(0)=g(1)$.



Define the discrete divergence $\text{div}\colon \mathbb{R}^{N\times M\times 2}\to \mathbb{R}^{N\times M}$ using the backwards differences,

$$ \text{div}(v)(n) := \delta_1^*v_1(n)+\delta_2^*v_2(n)$$.

Notice that this definition implies a discrete version of Green's theorem:
$$
\langle \nabla f, v  \rangle_{\mathbb{R}^{N\times M\times 2}}  
= - \, \langle f, \text{div}(v)  \rangle_{\mathbb{R}^{N\times M}}
$$


- **Exercise:** Let $x\in\mathbb{R}^n$ be a vector and $B\in\mathbb{R}^{n\times n}$ be a matrix. Let $B^\top$ be the matrix derived from $B$ by letting the $k$-th row of $B$ become the $k$-th column of $B^\top$. $B^\top$ is called the transpose of $B$. Show that
$$
\langle x,Bx\rangle = \langle B^{\top}x,x\rangle
$$
With this in mind it is reasonable to think of the gradient and the negative transpose of the divergence
$$
\nabla^{\top}=-\text{div}.
$$

Recall that the Laplace operator is defined as the sum of the second order partial derivatives

$$ \Delta f = \text{div}(\nabla f)$$

For our discrete images that means

$$
\Delta f(n_1,n_2) = f(n_1+1,n_2) + f(n_1,n_2+1) + f(n_1-1,n_2) + f(n_1,n_2-1) - 4f(n_1,n_2) .
$$

 - **Exercise:** This indeed approximates the continuous second derivative. For a smooth $g\colon\mathbb{R}\to\mathbb{R}$ you can try to show (using second order Taylor or otherwise) that
$$
\lim_{h\to 0 } \frac{f'(x+h)-2f(x)+f'(x-h)}{h^2} = f''(x).
$$
Again this shows that our discrete Laplace approximates $\frac{\partial^2f}{\partial x_1^2}+\frac{\partial^2f}{\partial x_2^2}$.



# Integral operators


Consider the total "energy" of the gradient vector field

$$J(f):=\frac{1}{2}\|\nabla f\|^2_2 := \frac{1}{2}\sum_n\|\nabla f(n)\|^2.$$

The operator $J$ maps a function to a number, here the total energy of the gradient vector field, i.e. it measures the smoothness of $J$. The more uniformly smooth $f$, the smaller $J(f)$ will be. If $f$ was defined on a continuous rather than a discrete domain we would replace the sum by an integral.

A related operator is the "total variation" defined as the summation of the individual gradient lengths

$$J_{TV}(f):= \|\nabla f\|_1 := \sum_n\|\nabla f(n)\|.$$


 - **Exercise:** What is the total variation and its geometric interpretation for the following image?
 
 $$
 \begin{pmatrix}
 0&0&0&0&0\\
 0&1&1&0&0\\
 0&1&1&1&0\\
 0&1&1&0&0\\
 0&0&0&0&0\\
 \end{pmatrix}
 $$

# Gradient flow PDE and Energy minimization

We now apply energy minimization to de-noise images. Consider the following image and its noisy version. Data aquired in real life with sensors or cameras is always corrupted by a certain amount of noise. Here we artificially create a noisy image and would like to recover the original image. One idea is to denoise by reducing the gradient energy.

In [8]:
im = convert(Array{Float64}, Gray.(load("chomsky.jpg")))
im = im[51:306, 1:256]
#im = convert(Array{Float64}, testimage("cameraman"))
M, N = size(im)
# Add some noise of variance sigma
sigma = 0.06  
randmat = randn(size(im));
im_noisy = im + sigma*randmat

Gray.([im im_noisy])

UndefVarError: UndefVarError: load not defined

## Gradient flow




The gradient of the scalar field $J$ is a vector field. Locally up to first order it describes the variation of $J$, i.e.

$$
J(f+\epsilon) = J(f) + \nabla J(f) \cdot \epsilon + o(\|\epsilon\|)
$$


Gradient descent should bring us towards a minimizer of $J$, i.e. we attempt to iterate

$$
f^{(k+1)}=f^{(k)}-\tau\nabla J(f^{(k)}).
$$

The latter equation can be seen as a finite difference approximation of the gradient flow partial differential equation (PDE) for a function $f=f(t,x)$ that depends on a continuous time parameter $t\geq 0$, namely

$$
\frac{\partial f}{\partial t} = - \nabla J(f) \quad\text{ with initial values }\quad f(0,\cdot)=f
$$

The discretization follows choosing the discrete time steps $t_k=k\tau.$
 
 
 
 - **Example:** As a simple case to compare with, here is an ordinary differential equation (ODE): Suppose $g(t)$ is a function of one variable $t\in\mathbb{R}$ only, and consider the ordinary differential equation $g'=kg$ with initial value $g(0)=g_0$. This simple equation can be directly solved for $g$ via separation of the variables as follows
 $$
 \frac{dg}{dt}=kg \quad\Longrightarrow\quad \frac{dg}{g}=kdt
 $$
 Integrating yields $\ln g=kt+c$ and thus $g(t)=Ce^{kt}$ where $g>0$. The constant $k$ is determined by the initial condition $g_0=g(0)=Ce^0=C$. By differentiating you can check that $g(t)=g_0e^{kt}$ indeed solves the differential equation. This was an ODE because $g$ depends only on one variable. A PDE is an equation which gives a relationship between the partial derivatives of a function of several independent variables and the gradient flow above is an example of a PDE.
 
 - **Exercise:** The equation of the previous example belongs to a more general class of ODE's, those with *seperate variables*. One can attempt to solve them by directly integrating as follows: If $g(y)\neq 0$ then
$$
y'=\frac{dy}{dx}=f(x)g(y)\quad\Longrightarrow
\quad \frac{dy}{g(y)}=f(x)\,dx
\quad\Longrightarrow
\quad \int\frac{dy}{g(y)}=\int f(x)\,dx
$$
leads to an implicit formula for $y$. In addition one may have solutions of the form $y=y_0$ namely if for some constant $y_0$ the equation $g(y_0)=0$ holds.
Find the solution to $y'=-\frac{x}{y}$ (where $y\neq 0$).



## The heat flow

- **Exercise:** Let $x\in\mathbb{R}^n$ be a vector and $B\in\mathbb{R}^{n\times n}$ be a matrix. Show that
$$
\nabla \langle x,Bx\rangle = B^{\top}x + Bx.
$$
(Hint: Show $\partial_k \langle x,Bx\rangle=(B^{\top}x)_k + (Bx)_k$) Use this to show that
$$
\nabla \frac{1}{2}\|Bx\|^2=B^\top Bx
$$
(Hint: What is $(B^\top B)^\top$)




For $J$ being the total gradient energy we therefore get 


$$ \nabla J(f) = -\Delta f$$

Thus in this case the gradient flow results precisely in a very famous PDE from physics: The heat equation

$$
\frac{\partial f}{\partial t} = \Delta f \quad\text{ with }\quad f(0,\cdot)=f
$$



In [9]:
# Parameters
tau = 1/4 
niter = 300

M, N =size(im);
Laplace = zeros(size(im))
m = 2:(M-1)
n = 2:(N-1)

# Gradient descent
f = im;     
for i = 1:niter
    Laplace[n,m] = f[n.-1,m] + f[n.+1,m] + f[n,m.-1] + f[n,m.+1] - 4*f[n,m];
    f = f + tau*Laplace;
end

Gray.([im f])

BoundsError: BoundsError: attempt to access ()
  at index [1]

## The total variation flow

Observe the effect of the heat flow. Noise would get blurred out and disappear, but so would be all important information in the image. The heat diffusion is isotropic and uniform. It does not regard any edges in the image. We would like to slow down the diffusion across the edges. That way one could hope to remove noise from an image without blurring edges.

We could use the magnitude of the gradient to produce anisotropic diffusion that preserves edges. Instead of using
$\Delta f=\text{div}(\nabla f)$ how about we only diffuse in the direction of the gradient if the gradient magnitude is small, i.e. not across strong edges. We could try to achieve this by using

$$
\text{div}\left(\frac{\nabla f}{\|\nabla f\|}\right)
$$
for all points for which $\|\nabla f\|\neq 0$. As it happens, this is the gradient flow for the total variation. That is, one can show,
$$
(\nabla J_{TV}(f))(n) = - \text{div}\left(\frac{\nabla f}{\|\nabla f\|}\right)(n)
$$
whenever $\nabla f(n)\neq 0$.

There is still the problem that the flow is unstable: Division by zero is not defined and wherever the gradient is very small, $(\nabla J_{TV}(f))(n)$ blows up. In many natural images, there will be regions where the gradient is zero or very small.
To get out of this trouble, consider a smoothed version of the total variation

$$J_{TV}^{\epsilon}(f)=\sum_n \sqrt{\epsilon^2+\|\nabla f(n)\|^2}$$


- ** Exercise:** To see what is going on here sketch for $s\in[-1,1]$ the functions $g(s)=|s|$ and $g_{\epsilon}(s)=\sqrt{\epsilon^2+s^2}$. What happens to $g_{\epsilon}(s)$ when $\epsilon\to 0$ ?


We want to use this stabilized TV energy, i.e.
$$ \nabla J_{TV}^{\epsilon}(f)=-\text{div}\left(\frac{\nabla f}{\sqrt{\epsilon^2+\|\nabla f\|^2}}\right)$$

Choosing a smaller $\epsilon$ makes the flow closer to a minimization of the total variation, but makes the computation unstable due to divisions by very small numbers. Try different $\epsilon$ in the following visiulaization.

In [10]:
epsilon = 0.01;

f = im;
M, N = size(im)
Laplace = zeros(size(im))
j = 1:(M-1)
k = 1:(N-1)
m = 2:(M-1)
n = 2:(N-1)

fdx = f[j.+1,k] - f[j,k]
fdy = f[j,k.+1] - f[j,k]
Gnorm = sqrt.(epsilon^2 .+ fdx.^2 .+ fdy.^2);
fdx = fdx ./ Gnorm
fdy = fdy ./ Gnorm;
div = fdx[n,m] - fdx[n.-1,m] + fdy[n,m] - fdy[n,m.-1];
    
Gray.(div)

BoundsError: BoundsError: attempt to access ()
  at index [1]

The smoothed total variation flow now is

$$
\frac{\partial f}{\partial t} = \text{div}\left(\frac{\nabla f}{\sqrt{\epsilon^2+\|\nabla f\|^2}}\right) 
\quad\text{ with } \quad f(0,\cdot)=f.
$$


In practice the flow is again being computed via gradient descent. For the smoothed total variation flow to converge, one needs to impose $\tau<\epsilon/4$. Thus being more faithful to the TV energy requires smaller time steps and thus yields a slower algorithm.

In [11]:
# Parameters:
epsilon= 0.006
tau = epsilon/5
niter = 400

M, N = size(im);
fdx = zeros(size(im))
fdy = zeros(size(im))
div = zeros(size(im))

j = 1:(M-1); k = 1:(N-1); m = 2:(M-1); n=2:(N-1)

# Gradient descent:
ftv = im
for i=1:niter
    fdx[j,k] = ftv[j.+1,k] - ftv[j,k]
    fdy[j,k] = ftv[j,k.+1] - ftv[j,k]
    Gnorm = sqrt.( epsilon^2 .+ fdx.^2 .+ fdy.^2)
    fdx = fdx ./ Gnorm
    fdy = fdy ./ Gnorm
    div[n,m] = fdx[n,m] - fdx[n.-1,m] + fdy[n,m] - fdy[n,m.-1]
    ftv += tau*div;
end
Gray.([im ftv])

BoundsError: BoundsError: attempt to access ()
  at index [1]

Now try denoising using gradient flow.

In [12]:
# Parameters:
epsilon = 0.0015
tau = epsilon/5
niter = 250

M,N = size(im)
fdx = zeros(size(im))
fdy = zeros(size(im))
div = zeros(size(im))
j = 1:(M-1); k = 1:(N-1); m = 2:(M-1); n = 2:(N-1)

# Gradient descent:
ftv = im_noisy
for i = 1:niter
    fdx[j,k] = ftv[j.+1,k] - ftv[j,k]
    fdy[j,k] = ftv[j,k.+1] - ftv[j,k]
    Gnorm = sqrt.(epsilon^2 .+ fdx.^2 .+ fdy.^2)
    fdx = fdx ./ Gnorm
    fdy = fdy ./ Gnorm
    div[n,m] = fdx[n,m] - fdx[n.-1,m] + fdy[n,m] - fdy[n,m.-1]
    ftv += tau*div
end

Gray.([im_noisy ftv])

BoundsError: BoundsError: attempt to access ()
  at index [1]

# PDE flow for inpainting


Data aquired from measurement devices like digital cameras or ccd's in microscopes or telescopes will always be corrupted, for example by random noise. Another possible corruption could be missing data, i.e. one only has scattered data of a function and needs to interpolate the function for the points where one didn't aquire measurements. Of course, for a totally arbitrary function the values of the function at those points could have been virtually anything and there is no hope of finding the true but unknown values. However, if we are dealing with natural images, we may reasonably assume they have small overall gradient energy or small total variation. This holds in particular for example for many medical images. 

Again, let us artificially create such a situation by randomly deleting a percentage of the pixels and suppose that this actually was the data we have to work with. How can we reconstruct the original from this corrupted image?

In [13]:
# Randomly knock off a certain percentage of the pixels
per = 0.8 #fraction of pixels to knock out
mask = rand(M,N)
mask[ mask.> per ] .= 1
mask[ mask.< 1 ]   .= 0
f = (mask).*im;


Gray.([im f])

UndefVarError: UndefVarError: M not defined

A key idea again is that whatever we fill into the missing pixels, the result should overall 
be rather smooth. That is, we are using our prior assumption that we are dealing with a 
natural image. We discovered earlier that the heat flow will converge to a minimizer for the overall gradient energy. So how about trying a heat flow. But: The heat flow will also affect the points where we actually know the true data. So in each iteration, let's just put the given data back. Then the heat diffusion will only flow into the missing pixels and leave our known pixels unchanged.

In [14]:
tau = 1/4
niter = 1000

M,N = size(im)
Laplace = zeros(size(im))
m = 2:(N-1); n=2:(M-1)

fheat = f
for i = 1:niter
    Laplace[n,m] = fheat[n.-1,m] + fheat[n.+1,m] + fheat[n,m.-1] + fheat[n,m.+1] - 4*fheat[n,m];
    fheat += tau*Laplace;
    fheat = fheat.*(mask.==0) + f; 
end

Gray.([im f fheat])

BoundsError: BoundsError: attempt to access ()
  at index [1]

Let us try the same with the total variation flow.

In [15]:
epsilon = 0.1
tau = epsilon/5
niter = 400

M,N = size(im)
fdx = zeros(size(im));  fdy = zeros(size(im)); div = zeros(size(im))
j = 1:(N-1); k = 1:(M-1); m = 2:(N-1); n = 2:(M-1)

#Gradient descent:
ftv = f;
for i = 1:niter
    fdx[j,k] = ftv[j.+1,k] - ftv[j,k]; fdy[j,k] = ftv[j,k.+1] - ftv[j,k]
    Gnorm = sqrt.( epsilon^2 .+ fdx.^2 .+ fdy.^2)
    fdx = fdx ./ Gnorm; fdy = fdy ./ Gnorm
    div[n,m] = fdx[n,m] - fdx[n.-1,m] + fdy[n,m] - fdy[n,m.-1]
    ftv += tau*div
    ftv = ftv.*(mask.==0) + f
end

Gray.([f fheat ftv])

BoundsError: BoundsError: attempt to access ()
  at index [1]

# Regularization for simultaneous denoising and impainting

Let's get a little more realistic. A certain amount of noise will always be there. So suppose we have to inpaint, but the measurements we have are also noisy. Now we would not want to put the noisy data back in each step of the iteration. Because we also want to denoise. Let's first create some artificial data.

In [16]:
# Add some noise
sigma    = 0.06  #Variance of the noise
randmat  = randn(size(im));
im_noisy = im + sigma*randmat

# Randomly knock off a certain percentage of the pixels
per = 0.15 #fraction of pixels to knock out
mask = rand(M,N)
mask[ mask.> per ] .= 1
mask[ mask.< 1 ]   .= 0
f = (mask).*im_noisy;


Gray.([im f])

UndefVarError: UndefVarError: M not defined

Let us try to model the situation we are in. First, to make our life easier let us assume our image was
just a single column
of pixels. So we are observing some vector $f\in\mathbb{R}^N$, one column of $N$ pixels. Assume further that our observation contains a certain small amount of random noise, i.e. there is a noise vector $\eta\in\mathbb{R}^N$. 
Finally, assume some pixels are just totally missing. We model this with a $N\times N$ matrix $P$ that is diagonal (all off-diagonal entries are zero) and that on the diagonal only has zeros and ones. If say 40% of the diagonal entries are zero, we are missing 40% of the data in our observation, e.g.

$$
\begin{pmatrix}
1&0&0&0&0 \\
0&0&0&0&0 \\
0&0&1&0&0 \\
0&0&0&1&0 \\
0&0&0&0&0 \\
\end{pmatrix}
\begin{pmatrix}
u_1\\
u_2\\
u_3\\
u_4\\
u_5\\
\end{pmatrix}
=
\begin{pmatrix}
u_1\\
0\\
u_3\\
u_4\\
0\\
\end{pmatrix}
$$


Our model now reads

$$
f=P u + \eta
$$

where $u$ is the unknow image we want to recover from observing the "corrupted" image $f$.

One strategy is to solve the following minimization problem

$$ \min\limits_{u} \frac{1}{2}\|f-P u\|^2+ \lambda J(u) $$

Here $\frac{1}{2}\|f-P u\|^2$ is called the data fidelity term. The intuition is that the noise $\eta$ is relatively small, thus the Euclidean distance between $f$ and $Pu$ should be small, but not zero. The second term $\lambda J(u) $
is called the regularization term. In principle anything could have been in the places where the pixels are missing. However we are assuming that the unknown image was a natural image. Thus it makes sense to look for $u$ with small gradient energy $J(u)$ and $\lambda$ is a parameter with which one can control how much a large gradient energy will be penalized. Other priors, say $J_{TV}^{\epsilon}$, can also be used as regularization.

- **Exercise:** We attempt to find $u$ via gradient descent, starting from some initial value $f^{(0)}$. Show that this leads to the iteration 
$$ f^{(k+1)} =f^{(k)} - \tau(P^\top(P f^{(k)}-f)
-\lambda\nabla J(f^{(k)})) $$

Note that in other applications there might be another degradation operator instead of $P$. For instance some operator that models how the image might be blurred due to camera shake, motion, or optics.

An important question is how to choose $\lambda$. Choosen too small not all noise may be removed, while choosing too large may result in destroying important feature of the image.

## Gradient energy regularization

There is an important small detail in the implementation of the gradient descent. Images are in fact matrices, not vectors. Thus the projection $P$ is not a diagonal matrix, but some matrix (not diagonal) with entries being either 0 or 1 and which is multiplied entrywise (sic) with the image $u$. A diagonal matrix coincides with its transpose matrix. Thus in the implementation we still keep $P^\top=P$, even though the matrix is not diagonal.

In [17]:
tau = 0.01
lambda = 3
niter = 500

M,N = size(im)
Laplace = zeros(size(im))
m = 2:(N-1); n = 2:(M-1)

fheat = f
for i = 1:niter
    Laplace[n,m] = fheat[n.-1,m] + fheat[n.+1,m] + fheat[n,m.-1] + fheat[n,m.+1] - 4*fheat[n,m]
    fheat = fheat - tau*mask.*(mask.*fheat-f) + lambda*tau*Laplace
end

Gray.([f fheat])

BoundsError: BoundsError: attempt to access ()
  at index [1]

## Total variation regularization

Now we compare gradient energy regularization and TV regularization, i.e. instead of the regularization $\lambda J(u)$ we now use $\lambda J_{TV}^{(\epsilon)}(u)$.

In [18]:
epsilon = 0.2
tau = epsilon/5
niter = 500
lambda  = 1

M,N = size(im)
fdx = zeros(size(im))
fdy = zeros(size(im))
div = zeros(size(im))
j = 1:(N-1); k = 1:(M-1); m = 2:(N-1); n=2:(M-1)

# Gradient descent:
ftv = f
for i = 1:niter
    fdx[j,k] = ftv[j.+1,k] - ftv[j,k]
    fdy[j,k] = ftv[j,k.+1] - ftv[j,k]
    Gnorm = sqrt.(epsilon^2 .+ fdx.^2 .+ fdy.^2)
    fdx = fdx ./ Gnorm
    fdy = fdy ./ Gnorm
    div[n,m] = fdx[n,m] - fdx[n.-1,m] + fdy[n,m] - fdy[n,m.-1]
    ftv = ftv - tau*mask.*(mask.*ftv-f) + lambda*tau*div
end

Gray.([f fheat ftv])

BoundsError: BoundsError: attempt to access ()
  at index [1]

# References and further reading


Numerical Analysis is a vast and vastly important subject. In the computer everything is discrete! Apparently, although this is a subject in its own, one needs a firm knowledge of the basics of classical real analysis.

Gradient descent is as old as the mountains but still one of the most important numerical algorithms.  For instance, it is the crucial piece of mathematics underlying the error backpropagation step when training neural networks in machine learning. Methods on how to accelerate its convergence therefore are subject of current research.  

The gradient energy sum/integral is connected to the name [Sergei Sobolev][3]. 
The total variation has been introduced to the area of signal processing by Rudin, [Osher][5] and Fetami. 
We will come back to the heat equation and its exact solution when we talk about the adventurous life and mathematics of the great French mathematician [Joseph Fourier][4], who started the field of harmonic analysis.

Large parts of the material of this notebook are based on the text: Stephane Mallat, *A wavelet tour of signal processing* (Academic Press 2008), which is available as electronic copy through the [NUS library][1].


[1]: http://linc.nus.edu.sg/search/
[3]: https://en.wikipedia.org/wiki/Sergei_Sobolev
[4]: https://en.wikipedia.org/wiki/Joseph_Fourier
[5]: https://en.wikipedia.org/wiki/Stanley_Osher
[6]: https://en.wikipedia.org/wiki/Unsharp_masking

